<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
!pip3 install folium
!pip3 install wget
!pip3 install pandas

In [2]:
import folium
import wget
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [4]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [8]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

# Loop through each launch site to add a circle and a marker
for index, row in launch_sites_df.iterrows():
    # Extract launch site coordinates
    coordinate = [row['Lat'], row['Long']]
    launch_site_name = row['Launch Site']
    
    # Create and add a circle at the launch site
    circle = folium.Circle(
        location=coordinate,
        radius=1000,  # Radius in meters
        color='#0000FF',
        fill=True,
        fill_color='#0000FF'
    ).add_child(folium.Popup(launch_site_name))
    
    # Create and add a marker with launch site name
    marker = folium.map.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launch_site_name,
        )
    )
    
    # Add circle and marker to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map
site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


Proximity to the Equator Line

The Equator is located at 0 degrees latitude.

After examining the map, it's clear that not all launch sites are near the Equator. While some launch sites, such as those at lower latitudes (e.g., Cape Canaveral in Florida), are relatively closer to the Equator compared to others, several sites are located at much higher latitudes (e.g., Vandenberg in California or Kennedy Space Center).

Explanation:

Launch sites closer to the Equator are advantageous because rockets launched from these sites can leverage Earth's rotational speed, which is highest at the Equator. This rotational boost reduces the fuel needed to reach orbit, making equatorial locations desirable for certain types of launches, especially geostationary orbits. However, polar orbit launches, which require different trajectories, benefit less from being near the Equator, so they can be placed at higher latitudes.
Proximity to the Coast
Yes, all launch sites are very close to the coast.

Explanation:

Coastal locations are chosen for safety reasons. Launching rockets over the ocean minimizes the risk to populated areas in case of launch failures. Additionally, if stages of the rocket fall back to Earth, they are more likely to land in the ocean, reducing the likelihood of debris hitting inhabited areas.

Summary of Findings:

Not all launch sites are close to the Equator, though some are situated in lower latitudes for efficiency in specific orbital launches.
All launch sites are located near the coast, primarily for safety and logistical reasons.







# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [9]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [10]:
marker_cluster = MarkerCluster()


_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [11]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

# Define a function to assign marker color based on launch success/failure
def assign_marker_color(launch_class):
    if launch_class == 1:
        return 'green'  # Success
    else:
        return 'red'    # Failure

# Create a new column in spacex_df to store the marker color
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

# Initialize a new map centered at NASA Johnson Space Center
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Create a MarkerCluster object
marker_cluster = MarkerCluster()

# Loop through each launch record and add markers to the marker cluster
for index, row in spacex_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    marker_color = row['marker_color']
    launch_site_name = row['Launch Site']
    
    # Create and add a marker for each launch record
    marker = folium.Marker(
        location=coordinate,
        icon=folium.Icon(color=marker_color),
        popup=f"Launch Site: {launch_site_name}\nSuccess: {row['class']}"
    )
    marker_cluster.add_child(marker)

# Add the marker cluster to the map
site_map.add_child(marker_cluster)

# Display the map
site_map

In [12]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [13]:
# Initialize a new map centered at NASA Johnson Space Center
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Create a MarkerCluster object
marker_cluster = MarkerCluster()

# Loop through each launch record and add a marker to the marker cluster
for index, row in spacex_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    marker_color = row['marker_color']
    launch_site_name = row['Launch Site']
    launch_outcome = "Success" if row['class'] == 1 else "Failure"
    
    # Create and add a marker with appropriate color and popup information
    marker = folium.Marker(
        location=coordinate,
        icon=folium.Icon(color=marker_color),
        popup=f"Launch Site: {launch_site_name}<br>Outcome: {launch_outcome}"
    )
    marker_cluster.add_child(marker)

# Add the marker cluster to the map
site_map.add_child(marker_cluster)

# Display the map
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [14]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [15]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [16]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

from math import sin, cos, sqrt, atan2, radians

# Define the calculate_distance function
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Launch site coordinates (Cape Canaveral)
launch_site_lat = 28.56342
launch_site_lon = -80.57674

# Coastline coordinates
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f"Distance from launch site to coastline: {distance_coastline:.2f} km")


Distance from launch site to coastline: 0.50 km


_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [17]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

# Coordinates of the launch site (Cape Canaveral)
launch_site_lat = 28.56342
launch_site_lon = -80.57674

# Coordinates of the closest coastline point
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate the distance using the provided function
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create a folium.Marker for the closest coastline point
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)

# Add the distance marker to the map
site_map.add_child(distance_marker)

# Display the map
site_map

_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [18]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)

# Coordinates of the launch site and closest coastline
launch_site_coordinates = [28.56342, -80.57674]
coastline_coordinates = [28.56367, -80.57163]

# Create a PolyLine connecting the launch site and the coastline
lines = folium.PolyLine(
    locations=[launch_site_coordinates, coastline_coordinates],
    weight=2,  # Line thickness
    color='blue'  # Line color
)

# Add the PolyLine to the map
site_map.add_child(lines)

# Display the map
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


In [19]:
# Coordinates of Cape Canaveral Launch Site
launch_site_coordinates = [28.56342, -80.57674]

# Coordinates of the closest city, railway, and highway
city_coordinates = [28.3922, -80.6077]
railway_coordinates = [28.5721, -80.5853]
highway_coordinates = [28.5634, -80.5707]

# Draw a line to the closest city
city_line = folium.PolyLine(
    locations=[launch_site_coordinates, city_coordinates],
    weight=2,
    color='green'
)
site_map.add_child(city_line)

# Draw a line to the closest railway
railway_line = folium.PolyLine(
    locations=[launch_site_coordinates, railway_coordinates],
    weight=2,
    color='red'
)
site_map.add_child(railway_line)

# Draw a line to the closest highway
highway_line = folium.PolyLine(
    locations=[launch_site_coordinates, highway_coordinates],
    weight=2,
    color='orange'
)
site_map.add_child(highway_line)

# Display the map
site_map

A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [20]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

# Coordinates of Cape Canaveral Launch Site
launch_site_coordinates = [28.56342, -80.57674]

# Coordinates of the closest city, railway, and highway (obtained using MousePosition)
city_coordinates = [28.3922, -80.6077]
railway_coordinates = [28.5721, -80.5853]
highway_coordinates = [28.5634, -80.5707]

# Calculate distances using the provided calculate_distance function
distance_city = calculate_distance(launch_site_coordinates[0], launch_site_coordinates[1],
                                   city_coordinates[0], city_coordinates[1])
distance_railway = calculate_distance(launch_site_coordinates[0], launch_site_coordinates[1],
                                      railway_coordinates[0], railway_coordinates[1])
distance_highway = calculate_distance(launch_site_coordinates[0], launch_site_coordinates[1],
                                      highway_coordinates[0], highway_coordinates[1])

# Create markers with distances and add them to the map
city_marker = folium.Marker(
    location=city_coordinates,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city)
    )
)
site_map.add_child(city_marker)

railway_marker = folium.Marker(
    location=railway_coordinates,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway)
    )
)
site_map.add_child(railway_marker)

highway_marker = folium.Marker(
    location=highway_coordinates,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway)
    )
)
site_map.add_child(highway_marker)

# Draw lines between the launch site and the markers
city_line = folium.PolyLine(
    locations=[launch_site_coordinates, city_coordinates],
    weight=2,
    color='green'
)
site_map.add_child(city_line)

railway_line = folium.PolyLine(
    locations=[launch_site_coordinates, railway_coordinates],
    weight=2,
    color='red'
)
site_map.add_child(railway_line)

highway_line = folium.PolyLine(
    locations=[launch_site_coordinates, highway_coordinates],
    weight=2,
    color='orange'
)
site_map.add_child(highway_line)

# Display the map
site_map

After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


Are launch sites in close proximity to railways?

Yes, launch sites are in close proximity to railways.

Explanation:

Railways are often located near launch sites for logistical reasons. Rockets and other heavy equipment are frequently transported by rail due to their large size and weight. Proximity to railways ensures efficient and safe transportation of rocket components from manufacturing facilities to launch sites.

Are launch sites in close proximity to highways?

Yes, launch sites are in close proximity to highways.

Explanation:

Highways provide easy access for personnel, equipment, and supplies required for launch operations. They also offer quick evacuation routes in case of emergencies.

Are launch sites in close proximity to the coastline?

Yes, launch sites are located very close to coastlines.

Explanation:

Launching rockets over the ocean reduces the risk to populated areas in case of launch failures or falling debris. Additionally, proximity to the ocean is crucial for launching satellites into specific orbits, especially geostationary orbits, which require an eastward trajectory over open water.

Do launch sites keep a certain distance away from cities?

Yes, launch sites maintain a significant distance from cities.

Explanation:

Launch sites are typically located in remote areas to ensure public safety. Rocket launches involve high risks, such as explosions or falling debris, and maintaining a safe distance from densely populated areas minimizes potential casualties and property damage.

Summary:

Launch sites are strategically positioned near coastlines, railways, and highways for safety, logistical efficiency, and accessibility.
They are deliberately kept far from cities to mitigate risks associated with rocket launches.
This proximity pattern ensures that launch operations are both efficient and safe, while minimizing the impact on civilian areas.

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
